## Injecting Data into PostgrelSQL


In [44]:
import os
from dotenv import load_dotenv
import psycopg2

In [45]:
load_dotenv()
password=os.getenv('Password')

In [46]:
def get_connection():
    return psycopg2.connect(
    host="localhost",
    port=5432,
    user="postgres",
    password=password,
    database="Inventory")


In [47]:
import pandas as pd
inventory_df=pd.read_csv('Data/Inventory.csv')
inventory_df.head(3)


,Unnamed: 0,date,sku_id,location_id,received_qty,sold_qty,spoiled_qty,closing_qty
0,0,2025-01-01,SKU_1,LOC_1,227,68,0,159
1,1,2025-01-01,SKU_1,LOC_2,415,124,0,291
2,2,2025-01-01,SKU_1,LOC_3,333,99,0,234


In [48]:
df_inven=inventory_df.columns[0]
inventory_df.drop(columns=df_inven,inplace=True)

In [49]:
inventory_df.head(5)

,date,sku_id,location_id,received_qty,sold_qty,spoiled_qty,closing_qty
0,2025-01-01,SKU_1,LOC_1,227,68,0,159
1,2025-01-01,SKU_1,LOC_2,415,124,0,291
2,2025-01-01,SKU_1,LOC_3,333,99,0,234
3,2025-01-01,SKU_1,LOC_4,206,61,0,145
4,2025-01-01,SKU_1,LOC_5,198,59,0,139


In [66]:
product_df=pd.read_csv('Data/Products.csv')
product_df.head(3)

,Unnamed: 0,sku_id,category,brand,unit,shelf_life,price
0,0,SKU_1,Dairy,Brand_9,litre,30,63.18
1,1,SKU_2,Home Care,Brand_2,litre,60,291.43
2,2,SKU_3,Beverages,Brand_1,litre,120,195.81


In [67]:
df_prod=product_df.columns[0]
product_df.drop(columns=df_prod,inplace=True)
product_df.head(3)

,sku_id,category,brand,unit,shelf_life,price
0,SKU_1,Dairy,Brand_9,litre,30,63.18
1,SKU_2,Home Care,Brand_2,litre,60,291.43
2,SKU_3,Beverages,Brand_1,litre,120,195.81


In [51]:
def query_table(sql_query):
    try:
    # Create connection
        conn = get_connection()

        results=pd.read_sql_query(sql_query,conn)

        return results

    except Exception as e:
        print(f"Error: {e}")
        return []

    finally:
        if cur:
            cur.close()
        if conn:
            conn.close()

In [52]:
def create_table(query):
    try:
        conn = get_connection()
        cur = conn.cursor()
        cur.execute(query)
        conn.commit()
        print("Table created.")
    except Exception as e:
        print(f"Create table error: {e}")
    finally:
        cur.close()
        conn.close()

In [53]:
def insert_value(df, query):
    df.columns = df.columns.str.strip().str.lower()

    try:
        conn = get_connection()
        cur = conn.cursor()

        for _, row in df.iterrows():
            values = tuple(row) 
            cur.execute(query, values)

        conn.commit()
        print("All rows inserted successfully.")
    except Exception as e:
        print(f"Insert error: {e}")
    finally:
        cur.close()
        conn.close()


In [63]:
create_table(''' create table Products(sku_id varchar(100),
             category varchar(150),brand varchar(100),unit varchar(50),shelf_life int,price float)''' )

Create table error: relation "products" already exists



In [68]:
insert_query=" INSERT INTO Products (sku_id, category, brand,unit, shelf_life, price) VALUES (%s, %s, %s, %s, %s, %s)"
insert_value(product_df,insert_query)

All rows inserted successfully.


In [65]:
print(product_df.columns)
print(product_df.shape)
print(tuple(product_df.iloc[0]))


Index(['category', 'brand', 'unit', 'shelf_life', 'price'], dtype='object')
(50, 5)
('Beverages', 'Brand_7', 'kg', np.int64(120), np.float64(375.24))


In [56]:
create_table('''CREATE TABLE Inventory(date DATE,sku_id varchar(100),
             location_id varchar(100),received_qty int,sold_qty int,spoiled_qty int,closing_qty int)''')

Table created.


In [57]:
insert_query2=''' INSERT INTO Inventory (date,sku_id,location_id,
                received_qty,sold_qty,spoiled_qty,closing_qty) VALUES (%s,%s,%s,%s,%s,%s,%s)'''
insert_value(inventory_df,insert_query2)

All rows inserted successfully.


In [58]:
df_sales=pd.read_csv('Data/Sales.csv')
df_sal_col=df_sales.columns[0]
df_sales.drop(columns=df_sal_col,inplace=True)
df_sales.head(3)

,order_id,sku_id,location_id,date,quantity,unit_price
0,SKU_1_LOC_1_20250101_0,SKU_1,LOC_1,2025-01-01,3,63.18
1,SKU_1_LOC_1_20250101_1,SKU_1,LOC_1,2025-01-01,5,63.18
2,SKU_1_LOC_1_20250101_2,SKU_1,LOC_1,2025-01-01,4,63.18


In [59]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254112 entries, 0 to 254111
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   order_id     254112 non-null  object 
 1   sku_id       254112 non-null  object 
 2   location_id  254112 non-null  object 
 3   date         254112 non-null  object 
 4   quantity     254112 non-null  int64  
 5   unit_price   254112 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 11.6+ MB


In [60]:
create_table(''' CREATE TABLE Sales (order_id varchar(100),sku_id varchar (100), location_id varchar(100),
              date DATE, quantity varchar(100), unit_price float);''')

Table created.


In [61]:
insert_query3=''' INSERT INTO Sales (order_id,sku_id,location_id,date,quantity,unit_price) 
                VALUES (%s,%s,%s,%s,%s,%s) '''
insert_value(df_sales,insert_query3)

All rows inserted successfully.


In [69]:
df_location=pd.read_csv('Data/Location.csv')
df_loc=df_location.columns[0]
df_location.drop(columns=df_loc,inplace=True)
df_location.head(3)

,location_id,zone,store_type,store_area
0,LOC_1,South,Convenience Store,4750
1,LOC_2,South,Hypermarket,3135
2,LOC_3,North,Supermarket,1061


In [70]:
create_table(''' CREATE TABLE Location (location_id varchar(100), zone varchar (100), store_type varchar(100),
              store_area varchar(100));''')

Table created.


In [72]:
insert_query4=''' INSERT INTO Location (location_id , zone , store_type,store_area) 
                VALUES (%s,%s,%s,%s) '''
insert_value(df_location,insert_query4)

All rows inserted successfully.


In [73]:
df_promotion=pd.read_csv('Data/Promotion.csv')
df_pro=df_promotion.columns[0]
df_promotion.drop(columns=df_pro,inplace=True)
df_promotion.head(3)

,sku_id,location_id,start_date,end_date,discount_percentage
0,SKU_1,LOC_5,2025-01-01,2025-01-04,18.0
1,SKU_3,LOC_2,2025-01-01,2025-01-04,23.0
2,SKU_6,LOC_2,2025-01-01,2025-01-04,14.0


In [74]:
create_table(''' CREATE TABLE Promotion (sku_id varchar(100),location_id varchar(100),
             start_date DATE, end_date DATE, discount_percentage float);''')

Table created.


In [75]:
insert_query5=''' INSERT INTO Promotion (sku_id,location_id,start_date, 
                end_date, discount_percentage) VALUES (%s,%s,%s,%s,%s) '''
insert_value(df_promotion,insert_query5)

All rows inserted successfully.


In [76]:
df_forecast=pd.read_csv('Data/Forecast.csv')
df_for=df_forecast.columns[0]
df_forecast.drop(columns=df_for,inplace=True)
df_forecast.head(3)

,date,sku_id,location_id,forecast_qty
0,2025-01-01,SKU_1,LOC_1,74
1,2025-01-01,SKU_1,LOC_2,132
2,2025-01-01,SKU_1,LOC_3,110


In [77]:
create_table(''' CREATE TABLE Forecast(date DATE,sku_id varchar(100),location_id varchar(100),forecast_qty int ) ''')

Table created.


In [78]:
insert_query6=''' INSERT INTO Forecast (date,sku_id,location_id,forecast_qty) VALUES (%s,%s,%s,%s)'''
insert_value(df_forecast,insert_query6)

All rows inserted successfully.
